In [1]:
!nvidia-smi

Mon Jun  3 10:20:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  | 00000000:81:00.0 Off |                    0 |
| N/A   30C    P0              53W / 400W |      4MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -q transformers datasets timm torch torchvision

In [12]:
from huggingface_hub import HfApi
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as transforms
from PIL import Image
from tqdm import tqdm
import timm

from datasets import load_dataset

In [3]:
# Load the dataset
dataset = load_dataset("mowoe/random-layouts")

# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def transform_examples(examples):
    examples["image"] = [transform(image.convert("RGB")) for image in examples["image"]]
    return examples

# Apply the transform_examples function to the dataset
dataset.set_transform(transform_examples)

In [4]:
# Debug: Check the structure of the dataset
print(type(dataset['train'][0]['image']))  # Should be <class 'torch.Tensor'>
print(dataset['train'][0]['image'].shape)  # Should be torch.Size([3, 224, 224])

<class 'torch.Tensor'>
torch.Size([3, 224, 224])


In [5]:
# Split dataset into train and test sets
train_size = int(0.8 * len(dataset['train']))
test_size = len(dataset['train']) - train_size
train_dataset, test_dataset = random_split(dataset['train'], [train_size, test_size])

In [6]:
# Custom collate function to handle batching
def collate_fn(batch):
    images = torch.stack([item['image'] for item in batch])
    labels = torch.tensor([item['label'] for item in batch], dtype=torch.long)
    return {'image': images, 'label': labels}

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

In [7]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained ResNet model
model = timm.create_model('resnet50', pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)  # Modify the classifier to output 2 classes
model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training loop
def train_model(model, train_loader, criterion, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch in tqdm(train_loader):
            images = batch['image'].to(device)
            labels = batch['label'].to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')


In [8]:
# Training the model
train_model(model, train_loader, criterion, optimizer, num_epochs=10)

  0%|          | 0/75 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:84.)
  return F.conv2d(input, weight, bias, self.stride,
100%|██████████| 75/75 [01:25<00:00,  1.14s/it]


Epoch [1/10], Loss: 0.5276


100%|██████████| 75/75 [01:21<00:00,  1.08s/it]


Epoch [2/10], Loss: 0.3847


100%|██████████| 75/75 [01:22<00:00,  1.10s/it]


...

Epoch [8/10], Loss: 0.0588


100%|██████████| 75/75 [01:21<00:00,  1.08s/it]


Epoch [9/10], Loss: 0.0430


100%|██████████| 75/75 [01:22<00:00,  1.09s/it]

Epoch [10/10], Loss: 0.0367


In [10]:
# Evaluate the model
from sklearn.metrics import accuracy_score

def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in tqdm(test_loader):
            images = batch['image'].to(device)
            labels = batch['label'].to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    print(f'Accuracy: {accuracy:.4f}')

In [11]:

# Evaluating the model
evaluate_model(model, test_loader)

100%|██████████| 19/19 [00:15<00:00,  1.21it/s]

Accuracy: 0.9550


In [13]:
api = HfApi()

In [14]:
from huggingface_hub import notebook_login

notebook_login()

In [15]:
torch.save(model.state_dict(), "./random-layout-classifier.pt")

In [16]:
api.create_repo("mowoe/random-layout-classifier",exist_ok=True,private=True, repo_type="model")
api.upload_file(
    path_or_fileobj="./random-layout-classifier.pt",
    path_in_repo="random-layout-classifier.pt",
    repo_id="mowoe/random-layout-classifier",
    repo_type="model",
)

random-layout-classifier.pt:   0%|          | 0.00/94.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mowoe/random-layout-classifier/commit/be25f21163666182a650c055e503aa0036e47e78', commit_message='Upload random-layout-classifier.pt with huggingface_hub', commit_description='', oid='be25f21163666182a650c055e503aa0036e47e78', pr_url=None, pr_revision=None, pr_num=None)